## First look at advective and diffuisive fluxes from model output

This notebook is a first look at the model's diffusive and advective fluxes form the diagnostics package.
Frequency of output is 43200 s (half a day).

Filename: Flux_TH (Theta), Flux_SLT (salinity)

| Variable   |   Units         | Description |
|------------|:---------------:|-------------:|
| ADVr_TH     | $^{\circ}C m^3s^{-1}$      | Vertical Advective Flux of Pot.Temperature|
| ADVx_TH     | $^{\circ}C m^3s^{-1}$      | Zonal Advective Flux of Pot.Temperature |
| ADVy_TH     | $^{\circ}C m^3s^{-1}$      | Meridional Advective Flux of Pot.Temperature|
| DFrE_TH     | $^{\circ}C m^3s^{-1}$      | Vertical Diffusive Flux of Pot.Temperature (Explicit part)|
| DFxE_TH     | $^{\circ}C m^3s^{-1}$      | Zonal      Diffusive Flux of Pot.Temperature|
| DFyE_TH     | $^{\circ}C m^3s^{-1}$      | Meridional Diffusive Flux of Pot.Temperature|
| DFrI_TH     | $^{\circ}C m^3s^{-1}$      | Vertical Diffusive Flux of Pot.Temperature (Implicit part)|   

Same for salinity.

For passive tracers, there are 3 extra fields besides these fluxes. Units correspond to DIC but it really doesn't matter.

Filename: FluxTR01, FluxTr02 

| Variable    |   Units                            | Description  |
|------------ |:---------------------------------: |-------------:|
| UTRAC01     |  $molC\ m^{-1}\cdot ms^{-1}$      |Zonal Mass-Weighted Transp of Tr01|
| VTRAC01     |  $molC\ m^{-1}\cdot ms^{-1}$      |Meridional Mass-Weighted Transp of Tr01|
| WTRAC01     |  $molC\ m^{-1}\cdot ms^{-1}$      |Vertical Mass-Weighted Transp of Tr01|
| ADVrTr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Vertical Advective Flux of Tr01|
| ADVxTr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Zonal Advective Flux of Tr01 |
| ADVyTr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Meridional Advective Flux of Tr01|
| DFrETr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Vertical Diffusive Flux of Tr01 (Explicit part)|
| DFxETr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Zonal      Diffusive Flux of Tr01|
| DFyETr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Meridional Diffusive Flux of Tr01|
| DFrITr01    |  $molC\ m^{-1}\cdot m^3s^{-1}$    | Vertical Diffusive Flux of Tr01 (Implicit part)|   
 


In [ ]:
#import gsw as sw # Gibbs seawater package

from math import *

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

%matplotlib inline

from MITgcmutils import rdmds

from netCDF4 import Dataset

import numpy as np

import os 

import pylab as pl

import scipy.io

import scipy as spy

import seaborn as sns

import sys

In [ ]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 
import TransportTools_MITgcm as trt

In [ ]:
sns.set()
sns.set_style('darkgrid')
sns.set_context('notebook')

In [ ]:
CanyonGrid='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_FluxDiagnostics/run01/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)
FluxTH = '/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/FluxTHGlob.nc'
FluxSLT = '/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/FluxTHGlob.nc'
FluxTR01 = '/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/FluxTHGlob.nc'
FluxTR01 = '/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/FluxTHGlob.nc'


In [ ]:
# General input

nx = 360
ny = 360
nz = 90

nt = 19 # t dimension size 

z = StateOut13.variables['Z']
#print(z[10])
Time = StateOut13.variables['T']
#print(Time[:])
xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

drF = CanyonGridOut.variables['drF'] # vertical distance between faces
dxG = rout.getField(CanyonGrid,'dxG')
bathy = rout.getField(CanyonGrid, 'Depth')

rA = rout.getField(CanyonGrid, 'rA') # area of cells (x-y)
hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid,'HFacC') # same for both runs


In [ ]:
Tr1Iso100 = rout.getField(CNTrun19Tr,'Tr1') # Tracer 1 CNT run19 , Kz = E-5
